# Metadata json to netCDF converter
Converting a jason metadata template file to a comtomized netCDF file, which contains only the header without data. 
From the template, users could add the attributes and change the value for each attribute for dimensions, global attributes and variables. 

In [1]:
# Import modules
import os 
import json
from netCDF4 import Dataset

In [2]:
# Define output nc/json filename
output_file = "empty_netcdf"

In [3]:
# Listing all the template JSON files in smpl/ directory
parentdir = os.getcwd()
jpath = os.path.join(parentdir, 'smpl') 
smpl_dir_list = os.listdir(jpath)
x=[print(i,smpl) for i, smpl in enumerate(smpl_dir_list)]

0 meta_tmpl.json
1 point_mooring_template.json
2 profile_ctd_template.json
3 profile_sonde_template.json
4 satellite_l2_template.json
5 satellite_L3_L4_template.json
6 trajectory_drifter_template.json


In [5]:
# Choose the template JSON file from the list by entering the index value.
ind = input("Choose the sample_json file by entering the index value: ")
# example_json = smpl_dir_list[int(ind)]
smpl_json = os.path.join(jpath,smpl_dir_list[int(ind)])
print(smpl_json)

Choose the sample_json file by entering the index value: 2
C:\Users\wenhaoli\Documents\GitHub\metadata-json2netcdf\smpl\profile_ctd_template.json


In [6]:
# A recursive function to change the attribute values.
def change_val(grp_attr):
    for key, value in grp_attr.items():
        if (type(value) is dict):
            #print("\n", "GROUP NAME: " + key)
            x = input("#### Want to change the attributes of '" + key + "'? (Y/Enter): ")
            if x.lower() == 'y':
                # Print the json file
                json_formatted_str = json.dumps(value, indent=2)
                print(json_formatted_str)
                change_val(value)
        else:
            #print ('\t', key, ' = ', value)
            x = input("--> Want to change the '" + key + "' value? (Y/Enter): ")
            if x.lower() == 'y':
                type_str = input("  -->> Enter the data type (int, float, double, or default('str')): ")
                if type_str ==  "int":
                    grp_attr[key] = int(input("Enter new value for '" + key + "': "))
                elif type_str ==  "float":
                    grp_attr[key] = float(input("Enter new value for '" + key + "': "))
                elif type_str ==  "double":
                    grp_attr[key] = float(input("Enter new value for '" + key + "': "))
                else:
                    grp_attr[key] = input("Enter new value for '" + key + "': ")
    return

In [8]:
# Read the json file
with open(smpl_json, 'r') as f:
    mData = json.loads(f.read())

In [9]:
# Print the sample json file
json_formatted_str = json.dumps(mData, indent=2)
print(json_formatted_str)

{
  "dimensions": {
    "depth": 79246,
    "time": 1
  },
  "variables": {
    "time": {
      "type": "double",
      "dimensions": [
        "time"
      ],
      "attributes": {
        "long_name": "Time of CTD measurement",
        "axis": "T",
        "standard_name": "time",
        "units": "days since 1950-01-01T00:00:00",
        "coverage_content_type": "coordinate"
      }
    },
    "latitude": {
      "type": "double",
      "dimensions": [
        "time"
      ],
      "attributes": {
        "_FillValue": -999,
        "long_name": "Latitude of CTD measurement",
        "valid_max": 16.6695,
        "valid_min": 16.6695,
        "axis": "Y",
        "standard_name": "latitude",
        "units": "degrees_north",
        "coverage_content_type": "coordinate"
      }
    },
    "longitude": {
      "type": "double",
      "dimensions": [
        "time"
      ],
      "attributes": {
        "_FillValue": -999,
        "long_name": "Longitude of CTD measurement",
        "

# Review and change "dimensions"

In [10]:
# dimensions fields
dims = mData['dimensions']
dims

{'depth': 79246, 'time': 1}

In [11]:
# Changing the key of dimensions
# dims['ni'] = dims.pop('lat')
# dims['nj'] = dims.pop('lon')

In [ ]:
# Loop through the dimensions, change the attribute value as needed.
change_val(dims)

# Review and change "global attributes"

In [ ]:
# Global attribute dict
glb_attr = mData['global_attributes']
glb_attr

In [ ]:
# Loop through the global attributes, change the attribute value as needed.
change_val(glb_attr)

In [ ]:
# Add new global attributes
isAdd_ga = input("-- Want to Add a new global attribute (Y/Enter): ")
while isAdd_ga.lower() == 'y':
    newKey = input("Enter the new key string: ")
    newAttr = input("Enter the new attribute value: ")
    glb_attr[newKey] = newAttr
    isAdd_ga = input("-- Want to Add another new global attribute (Y/Enter): ")
    if isAdd_ga.lower() != 'y': break

In [ ]:
json_formatted_str = json.dumps(glb_attr, indent=2)
print(json_formatted_str)

# Review and change "variables"

In [ ]:
# Variables group
var_attr = mData['variables']
#var_attr
print([key for key in var_attr.keys()])

In [ ]:
# Add the new variable to variables attribute
isAdd_var = input("-- Want to Add a new variable (Y/Enter): ")
while isAdd_var.lower() == 'y':
    newVar = input("Enter the new variable name: ")
    newValtmpl = input("Enter the template var name: ")
    var_attr[newVar]= var_attr[newValtmpl]
    isAdd_var = input("-- Want to Add another variable (Y/Enter): ")
    if isAdd_var.lower() != 'y': break

In [ ]:
# Remove the example_var variables
isDelvar = input("-- Want to delete the tmpl_var (Y/Enter): ")
while isDelvar.lower() == 'y':
    tmplVar = input("Enter the template var name to delete: ")
    if tmplVar in var_attr.keys(): 
        del var_attr[tmplVar]
    isDelvar = input("-- Want to delete the another tmpl_var (Y/Enter): ")
print([key for key in var_attr.keys()])

In [ ]:
# Loop through the variables
change_val(var_attr)

In [ ]:
# reviewing all the variables
# var_attr
json_formatted_str = json.dumps(var_attr, indent=2)
print(json_formatted_str)

# Write the new metadata to netCDF and json files to out_dir directory

In [ ]:
# Define the output netCDF file path
oPath = os.path.join(parentdir, 'out_dir') 
empty_nc = os.path.join(oPath, output_file + '.nc')
empty_nc

In [ ]:
if not os.path.exists(oPath):
    os.makedirs(oPath)

In [ ]:
# Create an empty netCDF file
with Dataset(empty_nc, "w") as nc:
    # Add dimensions
    for dim_name, dim_size in mData["dimensions"].items():
        nc.createDimension(dim_name, dim_size)

    # Add variables and their attributes
    for var_name, var_attrs in mData["variables"].items():
        var = nc.createVariable(var_name, var_attrs["type"], var_attrs["dimensions"])
        for attr_name, attr_value in var_attrs["attributes"].items():
            setattr(var, attr_name, attr_value)

    # Add global attributes
    for global_attr_name, global_attr_value in mData["global_attributes"].items():
        setattr(nc, global_attr_name, global_attr_value)

print(f"Empty netCDF file '{empty_nc}' created successfully with embedded JSON metadata attributes.")

In [ ]:
# system call to display the customized meta info from the netCDF. 
!ncdump -h out_dir\empty_netcdf.nc

In [ ]:
# Output the modified metadata dict to a json file.
out_json = os.path.join(oPath, 'meta_tmpl' + '.json')
with open(out_json, 'w', encoding='utf-8') as f:
    json.dump(mData, f, ensure_ascii=False, indent=4)